### Prueba de Cero Conocimiento Interactiva a partir del MPC SDitH

En este notebook vamos a convertir nuestra prueba de cero conocimiento interactiva que usa el protocolo MPC para SDitH, en una prueba de cero conociento no interactiva. La idea es omitir todos los pasos en que el probador interactua con el verificador, para esto se aplica la heurística de Fiat-Shamir. La idea es reemplazar todos los desafios que genera el verificador por la salida de funciones de hash asociadas a las computaciones realizadas hasta ese punto.

Este protocolo se describe en la siguiente secuencia de pasos:

- El probador genera las entradas compartidas aleatorias $[P], [Q], [a], [b], [c]$ y se compromete (_commit_) con las comparticiones de cada parte: 
  $[P]_i, [Q]_i, [a]_i, [b]_i, [c]_i \rightarrow \text{com}_i$, donde $i$ es el índice de la parte.

- Se deriva el primer desafío aplicando una función hash a los compromisos $\text{com}_1, \dots, \text{com}_N$ para obtener $h_1$ y se derivan los valores de aleatoriedad $r, \epsilon$ a partir de $h_1$.

- El probador simula el protocolo MPC usando las entradas compartidas $[P], [Q], [a], [b], [c]$, junto con la aleatoriedad $r, \epsilon$, y obtiene los valores $[\alpha], [\beta], [v]$.

- Se deriva el segundo desafío aplicando una función hash a $h_1$ y a los valores $[\alpha], [\beta], [v]$ para obtener $h_2$, que determina el subconjunto de partes $I$ que se deben abrir.

- El probador revela las comparticiones de entrada de todas las partes en el conjunto $I$: $[P]_i, [Q]_i, [a]_i, [b]_i, [c]_i$ para $i \in I$, junto con los compromisos $\text{com}_i$. También revela los valores $[\alpha]_i, [\beta]_i, [v]_i$ para las partes no abiertas $i \notin I$.

El verificador verifica la consistencia de los compromisos y de la computación MPC para las partes reveladas. Si todos los valores son consistentes, la firma es aceptada.

En este caso, el algoritmo de verificación se interpretaría de la siguiente manera:

- El verificador recalcula los compromisos para las partes $i \in I$, utilizando las comparticiones reveladas por el probador.
- Se deriva nuevamente el primer desafío aplicando una función hash a los compromisos recalculados e incluyendo los de las partes no abiertas, y se obtienen los valores de aleatoriedad $r, \epsilon$.
- El verificador simula el protocolo MPC para las partes $i \in I$, utilizando las entradas compartidas, los valores de los compromisos y la aleatoriedad $r, \epsilon$, para calcular los valores $[\alpha]_i, [\beta]_i, [v]_i$.
- Se deriva el segundo desafío aplicando otra función hash a los valores intermedios $(h_1, [\alpha], [\beta], [v])$ para verificar si se obtiene $h_2$, que es el valor que se incluyó en la firma.
- El verificador comprueba que los valores recalculados y los desafíos coinciden con los que forman parte de la firma. Si todos los valores son consistentes, la verificación es exitosa.

# Ejecución de la prueba de cero conocimiento

## 1. Configuración inicial

En este caso, el objetivo de nuestra prueba de cero conocimiento no interactiva es demostrar que conocemos una solución para el problema de decodificación del síndrome. Para esto se hace uso de tres polinomios $S,Q,P$ asociados a nuestra solución $x$ y un polinomio público $F$. Estas serán las entradas de cada una de las partes involucradas en el protocolo MPC.

En esta sección creamos los polinomios y creamos las particiones correspondientes.

In [1]:
import random
import hashlib

# Definir parámetros
definir_campo_q = 2  # Campo finito para el problema de decodificación del síndrome
F_q = GF(definir_campo_q)  # Campo finito de orden q
n = 6  # Longitud del vector de error
k = 3  # Longitud del vector de mensaje
m = n - k  # Número de filas de la matriz de paridad H

# Campo finito para los polinomios (debe ser mayor que F_q)
definir_campo_p = 17  # Campo finito para los polinomios, debe ser mayor que F_q y mayor que n
F_p = GF(definir_campo_p)  # Campo finito de orden p

# Definir el vector x en el campo F_q de longitud n
x = vector(F_q, [0, 0, 0, 0, 1, 1])  # Ejemplo con valores específicos

# Asociar los primeros n elementos del campo F_p con las coordenadas de x
f = [F_p(i) for i in range(n)]  # f_i para i en [1:n]

# Polinomio S: Interpolación de Lagrange usando los puntos (f_i, x_i)
R = F_p['X']
S = R.lagrange_polynomial([(f[i], x[i]) for i in range(n)])

# Polinomio Q: Producto de (X - f_i) para i en el subconjunto de índices con valor x_i = 1
X = R.gen()
E = [i for i in range(n) if x[i] != 0]  # Subconjunto de índices no nulos de x
Q = prod([(X - f[i]) for i in E])

# Polinomio F: Polinomio de "desaparición" para todos los f_i
F = prod([(X - f[i]) for i in range(n)])

# Polinomio P: Definido como P = S * Q / F
P = (S * Q).quo_rem(F)[0]  # División polinomial asegurando que F divide a S * Q sin residuo

# Evaluar los polinomios S y Q en los puntos f
evaluaciones_S = [S(f_i) for f_i in f]
evaluaciones_Q = [Q(f_i) for f_i in f]

# Imprimir los polinomios
def imprimir_polinomios():
    print("Polinomio S (Interpolación de Lagrange):")
    print(S)
    print("\nEvaluaciones de S en f:")
    print(evaluaciones_S)
    print("\nSubconjunto E:")
    print(E)
    print("\nPolinomio Q (Producto de raíces para índices no nulos de x):")
    print(Q)
    print("\nEvaluaciones de Q en f:")
    print(evaluaciones_Q)
    print("\nPolinomio F (Vanishing Polynomial):")
    print(F)
    print("\nPolinomio P (Resultado de S * Q / F):")
    print(P)
    
    print("\nEvaluación de Polinomios")
    print(f"\nS*Q==P*F: {S*Q==P*F}")

imprimir_polinomios()

Polinomio S (Interpolación de Lagrange):
13*X^5 + 11*X^4 + 10*X

Evaluaciones de S en f:
[0, 0, 0, 0, 1, 1]

Subconjunto E:
[4, 5]

Polinomio Q (Producto de raíces para índices no nulos de x):
X^2 + 8*X + 3

Evaluaciones de Q en f:
[3, 12, 6, 2, 0, 0]

Polinomio F (Vanishing Polynomial):
X^6 + 2*X^5 + 13*X^3 + 2*X^2 + 16*X

Polinomio P (Resultado de S * Q / F):
13*X + 4

Evaluación de Polinomios

S*Q==P*F: True


In [2]:
# Dividimos los polinomios S, Q y P en partes
n_parts = 5  # Número de partes

def dividir_polinomio_en_partes(polinomio, n_parts, campo):
    coeficientes = polinomio.coefficients(sparse=False)
    grado = polinomio.degree()
    partes = [[] for _ in range(n_parts)]  # Crear listas para cada parte

    for i in range(grado + 1):
        # Generamos aleatoriamente las primeras (n_parts - 1) partes en el campo
        partial_shares = [campo.random_element() for _ in range(n_parts - 1)]
        # Calculamos la última parte de manera que la suma de todas las partes sea igual al coeficiente original
        last_share = coeficientes[i] - sum(partial_shares)
        # Añadimos la última parte a la lista de partes
        partial_shares.append(last_share)
        # Distribuimos cada parte correspondiente entre las listas de partes de los participantes
        for j in range(n_parts):
            partes[j].append(partial_shares[j])
        
        # Imprimir las partes generadas para el coeficiente actual
        print(f"Coeficiente de x^{i} ({coeficientes[i]}): Partes -> {partial_shares}")

    return partes

# Función para generar los polinomios a partir de las partes
def construir_polinomios_de_partes(shares, n_parts, X):
    polinomios = []
    for j in range(n_parts):
        part_poly = sum(shares[j][i] * X^i for i in range(len(shares[j])))
        polinomios.append(part_poly)
    return polinomios

# Dividimos los polinomios S, Q y P en partes
print("\nDividiendo el polinomio S en partes:")
shares_S = dividir_polinomio_en_partes(S, n_parts, F_p)

print("\nDividiendo el polinomio Q en partes:")
shares_Q = dividir_polinomio_en_partes(Q, n_parts, F_p)

print("\nDividiendo el polinomio P en partes:")
shares_P = dividir_polinomio_en_partes(P, n_parts, F_p)

# Construimos los polinomios para cada una de las partes
def imprimir_polinomios_partes():
    partes_S_polinomios = construir_polinomios_de_partes(shares_S, n_parts, X)
    partes_Q_polinomios = construir_polinomios_de_partes(shares_Q, n_parts, X)
    partes_P_polinomios = construir_polinomios_de_partes(shares_P, n_parts, X)

    print("\nPolinomio que recibe cada parte para el polinomio S:")
    for j, part_poly in enumerate(partes_S_polinomios):
        print(f"Parte {j + 1} de S: {part_poly}")

    print("\nPolinomio que recibe cada parte para el polinomio Q:")
    for j, part_poly in enumerate(partes_Q_polinomios):
        print(f"Parte {j + 1} de Q: {part_poly}")

    print("\nPolinomio que recibe cada parte para el polinomio P:")
    for j, part_poly in enumerate(partes_P_polinomios):
        print(f"Parte {j + 1} de P: {part_poly}")

imprimir_polinomios_partes()

# Función para generar valores aleatorios en el campo
def generar_valores_aleatorios(campo, n_parts):
    valores = [campo.random_element() for _ in range(n_parts)]
    print(f"Valores aleatorios generados: {valores}")
    return valores

# Generar valores aleatorios para a_shares y b_shares
print("\nGenerando partes para a y b:")
a_shares = generar_valores_aleatorios(F_p, n_parts)
b_shares = generar_valores_aleatorios(F_p, n_parts)

# Calcular el producto c = a * b, y luego compartir el valor c
c = sum(a_shares) * sum(b_shares)
print(f"\nValor de c (producto de a y b): {c}")

# Generar partes para c_shares
c_shares = generar_valores_aleatorios(F_p, n_parts)
c_sum = sum(c_shares)
adjustment = c - c_sum
print(f"\nSuma de las partes de c antes del ajuste: {c_sum}")
print(f"Ajuste necesario para c: {adjustment}")

# Ajustar la primera parte de c_shares para garantizar que sum(c_shares) == c
c_shares[0] += adjustment
print(f"Partes ajustadas de c: {c_shares}")

print("\nResumen de lo que recibe cada parte:")
for i in range(n_parts):
    print(f"Parte {i + 1} recibe:")
    print(f"  - Coeficientes de S: {shares_S[i]}")
    print(f"  - Coeficientes de Q: {shares_Q[i]}")
    print(f"  - Coeficientes de P: {shares_P[i]}")
    print(f"  - Parte de a: {a_shares[i]}")
    print(f"  - Parte de b: {b_shares[i]}")
    print(f"  - Parte de c: {c_shares[i]}")


Dividiendo el polinomio S en partes:
Coeficiente de x^0 (0): Partes -> [16, 12, 16, 15, 9]
Coeficiente de x^1 (10): Partes -> [15, 12, 10, 9, 15]
Coeficiente de x^2 (0): Partes -> [14, 12, 0, 7, 1]
Coeficiente de x^3 (0): Partes -> [4, 5, 3, 12, 10]
Coeficiente de x^4 (11): Partes -> [9, 7, 4, 0, 8]
Coeficiente de x^5 (13): Partes -> [4, 1, 9, 12, 4]

Dividiendo el polinomio Q en partes:
Coeficiente de x^0 (3): Partes -> [6, 11, 16, 16, 5]
Coeficiente de x^1 (8): Partes -> [0, 1, 16, 4, 4]
Coeficiente de x^2 (1): Partes -> [9, 15, 7, 11, 10]

Dividiendo el polinomio P en partes:
Coeficiente de x^0 (4): Partes -> [15, 5, 11, 5, 2]
Coeficiente de x^1 (13): Partes -> [14, 16, 12, 2, 3]

Polinomio que recibe cada parte para el polinomio S:
Parte 1 de S: 4*X^5 + 9*X^4 + 4*X^3 + 14*X^2 + 15*X + 16
Parte 2 de S: X^5 + 7*X^4 + 5*X^3 + 12*X^2 + 12*X + 12
Parte 3 de S: 9*X^5 + 4*X^4 + 3*X^3 + 10*X + 16
Parte 4 de S: 12*X^5 + 12*X^3 + 7*X^2 + 9*X + 15
Parte 5 de S: 4*X^5 + 8*X^4 + 10*X^3 + X^2 +

### Generación de compromisos

Ya que se establecieron las entradas de cada parte del protocolo MPC que va a ejecutar el probador, se realiza un compromiso (_commit_) de cada una de las entradas.

In [3]:
# Función para obtener el commit de cada parte usando una función hash
def obtener_commit_de_parte_hash(shares_S, shares_Q, shares_P, a_shares, b_shares, c_shares, campo):
    commits = []
    valores_aleatorios = []
    for i in range(n_parts):
        # Generar un valor aleatorio para cada parte
        valor_aleatorio = campo.random_element()
        valores_aleatorios.append(valor_aleatorio)
        
        # Crear una cadena con todos los valores para la parte i
        datos_str = (
            f"{shares_S[i]}{shares_Q[i]}{shares_P[i]}"
            f"{a_shares[i]}{b_shares[i]}{c_shares[i]}"
            f"{valor_aleatorio}"
        )
        
        # Calcular el hash usando SHA-256
        commit = hashlib.sha256(datos_str.encode()).hexdigest()
        commits.append(commit)
        print(f"Commit de la parte {i + 1}: {commit} (valor aleatorio utilizado: {valor_aleatorio})")

    return commits, valores_aleatorios

# Obtener los commits de cada parte usando la función hash
print("\nObteniendo los commits de cada parte usando hash:")
commits, valores_aleatorios = obtener_commit_de_parte_hash(shares_S, shares_Q, shares_P, a_shares, b_shares, c_shares, F_p)


Obteniendo los commits de cada parte usando hash:
Commit de la parte 1: 58594a4814a62f23e0b8d373762f02966f8ba3c66d11e55ff6a7d7cbe43f5a2b (valor aleatorio utilizado: 5)
Commit de la parte 2: 7746252ccf0ac2cb1aa6a29ce1a642b9f1aca3c6faea96615227caa663f6b075 (valor aleatorio utilizado: 4)
Commit de la parte 3: 904d6bd6a328fff50ff116fd046c08474b08826e34dac3cf8022f386fe532ca3 (valor aleatorio utilizado: 13)
Commit de la parte 4: 62c7d16bbb3c8b5756e9240a49caa568b724b780bdeb4ecc007de3bf84e3e7bc (valor aleatorio utilizado: 16)
Commit de la parte 5: 71528b80a86bc1a8a43ae987cd52775b771bab7cc6614a6ae3c54b30dd65a857 (valor aleatorio utilizado: 7)


## 2. Derivación del primer desafío $h_1$ para crear $(r, \epsilon)$

Los valores de $(r, \epsilon)$ necesarios para ejecución del protocolo MPC comúnmente se obtienen de un verificador interesado en comprobar y conocer el resultado de la prueba, pero en este caso que se desea construir una prueba de cero conocimiento no interactiva no existe un verificador que esté en capacidad de suministrarnos dichos valores. Para esto, los valores se derivan de valores asociados a la ejecución del protocolo. En este caso primero se calcula el hash de los compromisos calculados para cada parte involucrada que va a participar en el protocolo y este hash se usa como semilla para la generación aleatoria de los valores.

In [4]:
# Paso 2: Derivación del primer desafío aplicando la función hash a los compromisos

# Concatenar todos los commits para formar una única cadena
commits_concatenados = ''.join(str(commit) for commit in commits)

# Calcular h1 aplicando SHA-256 a la cadena de commits concatenados
h1_hex = hashlib.sha256(commits_concatenados.encode()).hexdigest()
print(f"\nValor de h1 (hash de los compromisos concatenados): {h1_hex}")

# Convertir h1 de hexadecimal a un valor numérico para usarlo como semilla
h1_int = int(h1_hex, 16)

# Inicializar el generador aleatorio con la semilla h1
random.seed(h1_int)

# Convertir F_p a un entero para trabajar con random
F_p_int = int(F_p.order())  # Obtiene el orden del campo finito como un entero

# Derivar r y epsilon a partir del generador aleatorio
r = random.randint(1, F_p_int - 1)  # Generar un valor aleatorio r en el rango del campo finito
epsilon = random.randint(1, F_p_int - 1)  # Generar un valor aleatorio epsilon en el rango del campo finito

# Convertir r y epsilon a elementos del campo finito F_p
r = F_p(r)
epsilon = F_p(epsilon)

print(f"\nValores derivados de h1:")
print(f"r: {r}")
print(f"epsilon: {epsilon}")


Valor de h1 (hash de los compromisos concatenados): 2c3bd42907176019c4dc7c255abfe2bed458a38d42febce28fedebff6c1bd03e

Valores derivados de h1:
r: 10
epsilon: 1


## 3. Simulación del protocolo MPC en la cabeza

El probador simula el protocolo MPC "en su cabeza" usando las entradas compartidas y los valores de aleatoriedad $(r, \epsilon)$ generados en el paso anterior.

Durante esta simulación, el probador realiza las operaciones del protocolo MPC utilizando las comparticiones de entrada para calcular los valores intermedios **$\alpha$, $\beta$, $v$** para cada una de las partes. Estos valores intermedios serán utilizados para demostrar la validez del proceso sin revelar el valor secreto.

In [5]:
# Implementación del protocolo MPC

# Paso 2: Evaluar los polinomios S, Q, y P en un punto aleatorio r del campo finito
def evaluar_polinomios_en_punto(polinomios, r):
    return [part(r) for part in polinomios]

print(f"Valor aleatorio {r}")
print(f"Valuación en punto aleatorio {S(r)*Q(r)==P(r)*F(r)}")

evaluaciones_S_en_r = evaluar_polinomios_en_punto(construir_polinomios_de_partes(shares_S, n_parts, X), r)
evaluaciones_Q_en_r = evaluar_polinomios_en_punto(construir_polinomios_de_partes(shares_Q, n_parts, X), r)
evaluaciones_P_en_r = evaluar_polinomios_en_punto(construir_polinomios_de_partes(shares_P, n_parts, X), r)

# Paso 3: Calcular [alpha] y [beta] para cada parte
def calcular_alpha_beta(epsilon, evaluaciones_Q, evaluaciones_S, a_shares, b_shares):
    comparticion_alpha = [epsilon * evaluaciones_Q[j] + a_shares[j] for j in range(n_parts)]
    comparticion_beta = [evaluaciones_S[j] + b_shares[j] for j in range(n_parts)]
    return comparticion_alpha, comparticion_beta

comparticion_alpha, comparticion_beta = calcular_alpha_beta(epsilon, evaluaciones_Q_en_r, evaluaciones_S_en_r, a_shares, b_shares)

# Paso 4: Transmitir [alpha] y [beta] para obtener alpha y beta
def reconstruir_valor(comparticiones):
    return sum(comparticiones)

alpha = reconstruir_valor(comparticion_alpha)
beta = reconstruir_valor(comparticion_beta)
print(f"\nValor de alpha: {alpha}")
print(f"Valor de beta: {beta}")

# Paso 4: Calcular [v] para cada parte
# Calcular c como la multiplicación de la suma de a_shares y la suma de b_shares, luego dividir c en partes aleatorias
def calcular_v(epsilon, evaluaciones_P, a_shares, b_shares, alpha, beta, c_shares):
    comparticion_v = []
    for j in range(n_parts):
        if j == 0:
            # Para el primer elemento, restamos alpha * beta
            v_j = (epsilon * F(r) * evaluaciones_P[j]) - c_shares[j] + alpha * b_shares[j] + beta * a_shares[j] - alpha * beta
        else:
            # Para los demás elementos, no restamos alpha * beta
            v_j = (epsilon * F(r) * evaluaciones_P[j]) - c_shares[j] + alpha * b_shares[j] + beta * a_shares[j]
        comparticion_v.append(v_j)
    return comparticion_v

comparticion_v = calcular_v(epsilon, evaluaciones_P_en_r, a_shares, b_shares, alpha, beta, c_shares)

# Paso 5: Transmitir [v] para obtener v
v = reconstruir_valor(comparticion_v)
print(f"\nValor de v: {v}")

# Paso 7: Verificar si v es igual a 0
def verificar_v(v):
    if v == 0:
        print("Las partes aceptan: v = 0")
    else:
        print(f"Las partes rechazan: v = {v}")

verificar_v(v)

Valor aleatorio 10
Valuación en punto aleatorio True

Valor de alpha: 13
Valor de beta: 5

Valor de v: 0
Las partes aceptan: v = 0


## 4. Generación del Segundo Desafío

En el Paso 4, el objetivo es derivar el segundo desafío del protocolo no interactivo. Este desafío determina cuáles partes deben ser reveladas para la verificación de la consistencia del compromiso y la computación. Para ello, se utiliza el valor del primer desafío, $h_1$, junto con los valores intermedios obtenidos del protocolo MPC ($[\alpha], [\beta], [v]$). Se aplica una función hash a estos valores para generar un nuevo valor, $h_2$, que se utiliza como semilla para seleccionar aleatoriamente el subconjunto de partes, $I$, que deben ser abiertas. Este proceso asegura que el probador no tenga control sobre qué partes se revelarán, garantizando así la seguridad de la prueba de cero conocimiento.

In [6]:
# Paso 4: Derivación del segundo desafío y selección del subconjunto de partes a abrir

# Concatenar h1 y los valores de difusión alpha, beta, v para formar la entrada de la función hash
valores_intermedios_concatenados = (
    str(h1_int) + 
    ''.join(str(alpha_val) for alpha_val in comparticion_alpha) +
    ''.join(str(beta_val) for beta_val in comparticion_beta) +
    ''.join(str(v_val) for v_val in comparticion_v)
)

# Calcular h2 aplicando SHA-256 a los valores intermedios concatenados
h2_hex = hashlib.sha256(valores_intermedios_concatenados.encode()).hexdigest()
print(f"\nValor de h2 (hash de los valores intermedios concatenados): {h2_hex}")

# Convertir h2 de hexadecimal a un valor numérico para usarlo como semilla
h2_int = int(h2_hex, 16)

# Inicializar el generador aleatorio con la semilla h2
random.seed(h2_int)

# Determinar el subconjunto de partes I a abrir
# Vamos a seleccionar aleatoriamente la mitad de las partes para abrir
numero_partes_a_abrir = n_parts // 2
indices_revelados = random.sample(range(n_parts), numero_partes_a_abrir)

print(f"\nÍndices de las partes que se deben abrir: {indices_revelados}")


Valor de h2 (hash de los valores intermedios concatenados): 8d85367576a79025ce365209dea9afeee1d601c4fbb1d4fd111950317a10a45b

Índices de las partes que se deben abrir: [0, 1]


## 5. Revelación de valores 

En el Paso 5, el probador revela las comparticiones de entrada para las partes seleccionadas en el conjunto $I$ y los valos de $h_1$ y $h_2$. Esto incluye los valores compartidos de las entradas $[S]_i, [P]_i, [Q]_i, [a]_i, [b]_i, [c]_i$ para cada $i$ en $I$, junto con los compromisos $\text{com}_i$. Además, se revelan los valores de difusión $[\alpha]_i, [\beta]_i, [v]_i$ para las partes no abiertas. La revelación de estos valores permite al verificador comprobar la consistencia entre los compromisos, los valores calculados, y los valores intermedios. Esta verificación asegura que el probador esté siguiendo el protocolo correctamente y que la información presentada sea válida, lo cual es fundamental para mantener la integridad de la prueba de cero conocimiento.


In [24]:
# Paso 5: Revelación de las partes seleccionadas en el conjunto I

# Crear una lista ordenada para almacenar los valores de todas las partes
partes = []

print("\nRevelando las comparticiones para los índices seleccionados:")

# Extraer los valores correspondientes para cada índice y guardar en la lista 'partes'
for idx in range(n_parts):
    # Determinar si la parte está en el conjunto de las reveladas (I)
    es_revelada = idx in indices_revelados

    # Si la parte está revelada, guardamos todos los valores
    if es_revelada:
        parte = {
            "indice": idx,
            "es_revelada": True,
            "coef_S": shares_S[idx],
            "coef_Q": shares_Q[idx],
            "coef_P": shares_P[idx],
            "parte_a": a_shares[idx],
            "parte_b": b_shares[idx],
            "parte_c": c_shares[idx],
            "compromiso": commits[idx],
            "valor_aleatorio": valores_aleatorios[idx],
            "alpha": comparticion_alpha[idx],
            "beta": comparticion_beta[idx],
            "v": comparticion_v[idx],
        }

        partes.append(parte)

        # Imprimir información de la parte revelada
        print(f"\nParte {idx + 1} revelada:")
        for clave, valor in parte.items():
            if clave not in ["indice", "es_revelada"]:  # No imprimir el índice ni la bandera
                print(f"  - {clave}: {valor}")

    # Si la parte no está revelada, solo guardamos los valores necesarios para la verificación
    else:
        parte = {
            "indice": idx,
            "es_revelada": False,
            "alpha": comparticion_alpha[idx],
            "beta": comparticion_beta[idx],
            "v": comparticion_v[idx],
            "compromiso": commits[idx],
        }

        partes.append(parte)

        # Imprimir información de la parte no revelada
        print(f"\nParte {idx + 1} no revelada:")
        for clave, valor in parte.items():
            if clave not in ["indice", "es_revelada"]:  # No imprimir el índice ni la bandera
                print(f"  - {clave}: {valor}")
                
print(f"\nValor de h_1: \n{h1_hex}")
print(f"\nValor de h_2: \n{h2_hex}")


Revelando las comparticiones para los índices seleccionados:

Parte 1 revelada:
  - coef_S: [16, 15, 14, 4, 9, 4]
  - coef_Q: [6, 0, 9]
  - coef_P: [15, 14]
  - parte_a: 3
  - parte_b: 0
  - parte_c: 14
  - compromiso: 58594a4814a62f23e0b8d373762f02966f8ba3c66d11e55ff6a7d7cbe43f5a2b
  - valor_aleatorio: 5
  - alpha: 8
  - beta: 16
  - v: 8

Parte 2 revelada:
  - coef_S: [12, 12, 12, 5, 7, 1]
  - coef_Q: [11, 1, 15]
  - coef_P: [5, 16]
  - parte_a: 4
  - parte_b: 1
  - parte_c: 0
  - compromiso: 7746252ccf0ac2cb1aa6a29ce1a642b9f1aca3c6faea96615227caa663f6b075
  - valor_aleatorio: 4
  - alpha: 12
  - beta: 9
  - v: 6

Parte 3 no revelada:
  - alpha: 13
  - beta: 15
  - v: 5
  - compromiso: 904d6bd6a328fff50ff116fd046c08474b08826e34dac3cf8022f386fe532ca3

Parte 4 no revelada:
  - alpha: 4
  - beta: 2
  - v: 9
  - compromiso: 62c7d16bbb3c8b5756e9240a49caa568b724b780bdeb4ecc007de3bf84e3e7bc

Parte 5 no revelada:
  - alpha: 10
  - beta: 14
  - v: 6
  - compromiso: 71528b80a86bc1a8a43ae987cd

# Verificación de la Prueba de Cero Conocimiento

La verificación de la prueba de cero conocimiento consiste en comprobar la consistencia de los valores revelados por el probador. Primero, el verificador recalcula los compromisos de las partes reveladas utilizando las comparticiones obtenidas. Luego, verifica que el primer desafío, $h_1$, y los valores de difusión $[\alpha]_i, [\beta]_i, [v]_i$ coincidan con los que fueron calculados originalmente. También se deriva nuevamente el segundo desafío, $h_2$, para confirmar que el subconjunto de partes seleccionadas, $I$, es consistente con la firma. Si todos los valores coinciden y son válidos, la prueba se considera correcta y el verificador la acepta; de lo contrario, la rechaza, asegurando así que la integridad de la información presentada es confiable.


## 1. Recalcular compromisos de las partes reveladas

En el Paso 1 del algoritmo de verificación, el verificador recalcula los compromisos de las partes que han sido reveladas por el probador. 

Para cada parte seleccionada en el conjunto $I$, el verificador toma las comparticiones reveladas (incluyendo las entradas originales y los valores aleatorios) y las utiliza para recalcular el compromiso aplicando la misma función hash que se usó en la construcción original. 

El verificador luego compara cada compromiso recalculado con el compromiso que el probador presentó inicialmente. Si los compromisos coinciden, se valida que las comparticiones reveladas son consistentes con los valores comprometidos originalmente. Si algún compromiso no coincide, la verificación falla y la prueba es rechazada.

In [15]:
# Paso 1: Recalcular los compromisos de las partes reveladas

print("\nVerificando los compromisos de las partes reveladas...")

compromisos_validos = True

# Iterar sobre todas las partes almacenadas en la lista 'partes'
for parte in partes:
    # Solo verificamos las partes reveladas
    if parte["es_revelada"]:
        # Obtener los valores de las entradas reveladas
        coef_S = parte["coef_S"]
        coef_P = parte["coef_P"]
        coef_Q = parte["coef_Q"]
        parte_a = parte["parte_a"]
        parte_b = parte["parte_b"]
        parte_c = parte["parte_c"]
        valor_aleatorio = parte["valor_aleatorio"]

        # Recalcular el compromiso usando una función hash
        entrada_compromiso = f"{coef_S}{coef_Q}{coef_P}{parte_a}{parte_b}{parte_c}{valor_aleatorio}"
        compromiso_recalculado = hashlib.sha256(entrada_compromiso.encode()).hexdigest()

        # Obtener el compromiso original
        compromiso_original = parte["compromiso"]

        # Comparar el compromiso recalculado con el compromiso original
        if compromiso_recalculado != compromiso_original:
            print(f"Error: El compromiso recalculado no coincide para la parte {parte['indice'] + 1}.")
            compromisos_validos = False
        else:
            print(f"El compromiso recalculado coincide para la parte {parte['indice'] + 1}.")

# Determinar si todos los compromisos fueron verificados correctamente
if compromisos_validos:
    print("\nTodos los compromisos de las partes reveladas son válidos.")
else:
    print("\nAlgunos compromisos de las partes reveladas no son válidos. Verificación fallida.")



Verificando los compromisos de las partes reveladas...
El compromiso recalculado coincide para la parte 1.
El compromiso recalculado coincide para la parte 2.

Todos los compromisos de las partes reveladas son válidos.


## 2. Derivación del primer desafío $h_1$ durante la verificación

En el **Paso 2** del algoritmo de verificación, el verificador **recalcula el primer desafío** $h_1$ a partir de los compromisos obtenidos.

Para ello, el verificador utiliza todos los compromisos, tanto de las partes reveladas como de las no reveladas, y los **concatena** siguiendo el mismo orden que se utilizó durante la fase de construcción de la prueba. Luego, el verificador aplica una función hash para derivar el valor **$h_1$ de verificación**. 

Este paso es esencial para garantizar la **determinismo** del protocolo y para asegurar que la aleatoriedad utilizada en la construcción y verificación sea **idéntica**, permitiendo una reproducción precisa de la prueba.

In [16]:
# Paso 2: Derivar el primer desafío h1 durante la verificación (nueva derivación con nombre diferente)

# Concatenar todos los compromisos de las partes (en el orden original) para formar una única cadena
compromisos_concatenados = ''

# Iterar sobre todas las partes para concatenar los compromisos en orden
for parte in partes:
    compromisos_concatenados += parte["compromiso"]

# Calcular h1_verificacion aplicando SHA-256 a la cadena de compromisos concatenados
h1_verificacion_hex = hashlib.sha256(compromisos_concatenados.encode()).hexdigest()
print(f"\nValor de h1_verificacion (hash de los compromisos concatenados): {h1_verificacion_hex}")

# Convertir h1_verificacion de hexadecimal a un valor numérico para usarlo como semilla
h1_verificacion_int = int(h1_verificacion_hex, 16)

# Inicializar el generador aleatorio con la semilla h1_verificacion
random.seed(h1_verificacion_int)

# Derivar r y epsilon a partir del generador aleatorio
F_p_int = int(F_p.order())  # Obtiene el orden del campo finito como un entero
r = random.randint(1, F_p_int - 1)  # Generar un valor aleatorio r en el campo finito F_p
epsilon = random.randint(1, F_p_int - 1)  # Generar un valor aleatorio epsilon en el campo finito F_p

# Convertir r y epsilon a elementos del campo finito F_p
r = F_p(r)
epsilon = F_p(epsilon)

print(f"\nValores derivados de h1_verificacion durante la verificación:")
print(f"r: {r}")
print(f"epsilon: {epsilon}")


Valor de h1_verificacion (hash de los compromisos concatenados): 2c3bd42907176019c4dc7c255abfe2bed458a38d42febce28fedebff6c1bd03e

Valores derivados de h1_verificacion durante la verificación:
r: 10
epsilon: 1


## 3. Simulación del protocolo MPC para las partes seleccionadas en $I$

En el **Paso 3** del algoritmo de verificación, el verificador realiza una **simulación del protocolo MPC** para las partes que han sido seleccionadas en el conjunto $I$.

Para cada parte en el conjunto $I$, el verificador utiliza los valores de las entradas compartidas y los valores de aleatoriedad $(r, \epsilon)$ recalculados a partir de $h_1$. Luego, el verificador simula la ejecución del protocolo MPC, utilizando estos valores para **recalcular los valores intermedios** $\alpha$, $\beta$, y $v$.

Este paso garantiza que los valores calculados por el verificador sean consistentes con los valores originalmente generados por el probador. Si hay discrepancias, se indicará que la verificación ha fallado.

In [17]:
# Paso 3: Calcular [alpha] y [beta] para cada parte revelada y verificar si coinciden

print("\nCalculando y verificando [alpha] y [beta] para las partes reveladas...")

# Lista para almacenar el resultado de la verificación de cada parte
resultados_verificacion = []

# Iterar sobre todas las partes en la lista 'partes'
for parte in partes:
    # Solo procesar las partes reveladas
    if parte["es_revelada"]:
        # Obtener los valores necesarios
        coef_S = parte["coef_S"]
        coef_Q = parte["coef_Q"]
        coef_P = parte["coef_P"]
        parte_a = parte["parte_a"]
        parte_b = parte["parte_b"]
        valor_aleatorio = parte["valor_aleatorio"]

        # Calcular la evaluación del polinomio S y Q en el punto r
        evaluacion_S = sum(coef * (r ** i) for i, coef in enumerate(coef_S))
        evaluacion_Q = sum(coef * (r ** i) for i, coef in enumerate(coef_Q))

        # Calcular los valores de alpha y beta
        alpha_recalculado = epsilon * evaluacion_Q + parte_a
        beta_recalculado = evaluacion_S + parte_b

        # Obtener los valores originales para comparar
        alpha_original = parte["alpha"]
        beta_original = parte["beta"]

        # Verificar si los valores recalculados coinciden con los originales
        alpha_valido = (alpha_recalculado == alpha_original)
        beta_valido = (beta_recalculado == beta_original)

        # Guardar el resultado de la verificación
        resultados_verificacion.append((parte["indice"], alpha_valido, beta_valido))

        # Imprimir el resultado de la verificación para esta parte
        print(f"\nParte {parte['indice'] + 1}:")
        print(f"  Alpha recalculado: {alpha_recalculado}, Alpha original: {alpha_original}")
        print("  Alpha - Comparación: " + ("Iguales" if alpha_valido else "Diferentes"))
        print(f"  Beta recalculado: {beta_recalculado}, Beta original: {beta_original}")
        print("  Beta - Comparación: " + ("Iguales" if beta_valido else "Diferentes"))

# Determinar si todos los valores de [alpha] y [beta] fueron verificados correctamente
alpha_beta_validos = all(alpha_valido and beta_valido for _, alpha_valido, beta_valido in resultados_verificacion)

if alpha_beta_validos:
    print("\nTodos los valores de [alpha] y [beta] para las partes reveladas son válidos.")
else:
    print("\nAlgunos valores de [alpha] y [beta] para las partes reveladas no son válidos. Verificación fallida.")


Calculando y verificando [alpha] y [beta] para las partes reveladas...

Parte 1:
  Alpha recalculado: 8, Alpha original: 8
  Alpha - Comparación: Iguales
  Beta recalculado: 16, Beta original: 16
  Beta - Comparación: Iguales

Parte 2:
  Alpha recalculado: 12, Alpha original: 12
  Alpha - Comparación: Iguales
  Beta recalculado: 9, Beta original: 9
  Beta - Comparación: Iguales

Todos los valores de [alpha] y [beta] para las partes reveladas son válidos.


## 4. Derivación del segundo desafío $h_2$ durante la verificación

En el **Paso 4** del algoritmo de verificación, el verificador deriva el segundo desafío **$h_2$**.

Para ello, el verificador concatena el valor de $h_1$ de verificación junto con los valores intermedios $\alpha$, $\beta$, y $v$ para **todas las partes** (incluidas las reveladas y no reveladas). Luego, se aplica una función hash a esta concatenación para obtener el valor **$h_2$**.

Este valor se utiliza como semilla para seleccionar nuevamente el conjunto $I$ de índices de partes que deben ser verificadas. Si el conjunto seleccionado coincide con el conjunto original, se garantiza la consistencia del proceso.

In [30]:
# Paso 4: Derivar el segundo desafío h2_verificacion durante la verificación

# Concatenar h1_verificacion y los valores de difusión (alpha, beta, v) para formar la entrada de la función hash
valores_intermedios_concatenados = (
    str(h1_verificacion_int) +  # Concatenar el valor de h1_verificacion como un entero
    ''.join(str(parte["alpha"]) for parte in partes) +  # Concatenar todos los valores de alpha
    ''.join(str(parte["beta"]) for parte in partes) +  # Concatenar todos los valores de beta
    ''.join(str(parte["v"]) for parte in partes)  # Concatenar todos los valores de v
)

# Calcular h2_verificacion aplicando SHA-256 a la cadena de valores intermedios concatenados
h2_verificacion_hex = hashlib.sha256(valores_intermedios_concatenados.encode()).hexdigest()
print(f"\nValor de h2_verificacion (hash de los valores intermedios concatenados): {h2_verificacion_hex}")

# Convertir h2_verificacion de hexadecimal a un valor numérico para usarlo como semilla
h2_verificacion_int = int(h2_verificacion_hex, 16)


Valor de h2_verificacion (hash de los valores intermedios concatenados): 8d85367576a79025ce365209dea9afeee1d601c4fbb1d4fd111950317a10a45b


## 5. Verificación final y decisión de aceptación o rechazo

En el **Paso 5** del algoritmo de verificación, el verificador realiza la **verificación final** de los valores generados, en este caso la comparación se reduce a comparar los desafios intermedios $h_1$ y $h_2$.


In [29]:
# Paso final: Verificación y Decisión de Aceptación o Rechazo de la Prueba de Cero Conocimiento

print("\nVerificando las partes seleccionadas en el subconjunto I...")

# Comparar los valores de h1 y h2 originales con los recalculados durante la verificación
h1_valido = (h1_hex == h1_verificacion_hex)
h2_valido = (h2_hex == h2_verificacion_hex)

# Imprimir los resultados de las comparaciones de h1 y h2
print("\nComparación de valores hash:")
print(f"  Valor original de h1: {h2_hex}, Valor verificado de h1: {h1_verificacion_hex}")
print("  h1 - Comparación: " + ("Iguales" if h1_valido else "Diferentes"))

print(f"  Valor original de h2: {h2_hex}, Valor verificado de h2: {h2_verificacion_hex}")
print("  h2 - Comparación: " + ("Iguales" if h2_valido else "Diferentes"))

# Verificar si todos los compromisos son válidos para las partes seleccionadas
if compromisos_validos and alpha_beta_validos and h1_valido and h2_valido:
    print("\nVerificación exitosa: La prueba de cero conocimiento ha sido aceptada.")
else:
    print("\nVerificación fallida: La prueba de cero conocimiento ha sido rechazada.")


Verificando las partes seleccionadas en el subconjunto I...

Comparación de valores hash:
  Valor original de h1: 8d85367576a79025ce365209dea9afeee1d601c4fbb1d4fd111950317a10a45b, Valor verificado de h1: 2c3bd42907176019c4dc7c255abfe2bed458a38d42febce28fedebff6c1bd03e
  h1 - Comparación: Iguales
  Valor original de h2: 8d85367576a79025ce365209dea9afeee1d601c4fbb1d4fd111950317a10a45b, Valor verificado de h2: 8d85367576a79025ce365209dea9afeee1d601c4fbb1d4fd111950317a10a45b
  h2 - Comparación: Iguales

Verificación exitosa: La prueba de cero conocimiento ha sido aceptada.
